# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [7]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [8]:
list_params_alpha = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]
list_params_beta = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [9]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100
validation_training_ratio = 0.95

# Pipeline for Croston_TSB

In [10]:
def CrostonTSB_output_all_params(sales_pattern, list_params_alpha, list_params_beta, 
                                 start_train_date, end_train_date, n_pred_days, 
                                 num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params_alpha:
            for j in list_params_beta:
                predictions = list()
                history = [x for x in train_first[num_train_needed*-1:]]
                for t in range(len(train_second)):
                    yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                    predictions.append(yhat)
                    history.append(train_second[t])
                df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                               'Actual Data': train_second,
                                               'Forecast': predictions,
                                               'Alpha': [i for count_i in range(len(train_second))],
                                               'Beta': [j for count_j in range(len(train_second))]})
                df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [11]:
start = timer()

pattern_df_result_all_params = CrostonTSB_output_all_params(sales_erratic,
                                                          list_params_alpha,
                                                          list_params_beta,
                                                          start_train_date,
                                                          end_train_date, 
                                                          n_pred_days,
                                                          num_train_needed,
                                                          validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 12.257488355949999 minutes


In [12]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,FOODS_1_021_CA_1_validation,7.0,3.429785,0.1,0.1
1,FOODS_1_021_CA_1_validation,6.0,3.737938,0.1,0.1
2,FOODS_1_021_CA_1_validation,2.0,3.940782,0.1,0.1
3,FOODS_1_021_CA_1_validation,3.0,3.786177,0.1,0.1
4,FOODS_1_021_CA_1_validation,4.0,3.726393,0.1,0.1
...,...,...,...,...,...
740254,FOODS_3_809_CA_4_validation,0.0,1.083658,0.9,0.9
740255,FOODS_3_809_CA_4_validation,0.0,0.108366,0.9,0.9
740256,FOODS_3_809_CA_4_validation,0.0,0.010837,0.9,0.9
740257,FOODS_3_809_CA_4_validation,1.0,0.001084,0.9,0.9


In [13]:
pattern_df_result_all_params.to_csv('CrostonTSB_Erratic_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [14]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [15]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.8050344426499995 minutes


In [16]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_2_validation,0.1,0.1,0.891441,64.5906,66.3130,96.8901,4.556254,5.438849
1,FOODS_1_004_CA_2_validation,0.1,0.2,0.966713,70.0445,68.5639,105.8192,4.940975,5.933753
2,FOODS_1_004_CA_2_validation,0.1,0.3,0.995130,72.1035,69.5219,108.6595,5.086221,6.136172
3,FOODS_1_004_CA_2_validation,0.1,0.4,1.023644,74.1695,70.9047,111.7847,5.231959,6.292970
4,FOODS_1_004_CA_2_validation,0.1,0.5,1.054839,76.4298,72.7780,115.3252,5.391397,6.434534
...,...,...,...,...,...,...,...,...,...
20002,FOODS_3_827_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
20003,FOODS_3_827_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
20004,FOODS_3_827_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
20005,FOODS_3_827_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


# Check what products has unexpected metrics result

In [17]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2754,FOODS_1_163_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
2755,FOODS_1_163_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2756,FOODS_1_163_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
2757,FOODS_1_163_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
2758,FOODS_1_163_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
20002,FOODS_3_827_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
20003,FOODS_3_827_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
20004,FOODS_3_827_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
20005,FOODS_3_827_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [18]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2754,FOODS_1_163_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
2755,FOODS_1_163_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2756,FOODS_1_163_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
2757,FOODS_1_163_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
2758,FOODS_1_163_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
20002,FOODS_3_827_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
20003,FOODS_3_827_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
20004,FOODS_3_827_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
20005,FOODS_3_827_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [19]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2754,FOODS_1_163_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
2755,FOODS_1_163_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2756,FOODS_1_163_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
2757,FOODS_1_163_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
2758,FOODS_1_163_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
20002,FOODS_3_827_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
20003,FOODS_3_827_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
20004,FOODS_3_827_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
20005,FOODS_3_827_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [20]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2754,FOODS_1_163_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
2755,FOODS_1_163_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2756,FOODS_1_163_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
2757,FOODS_1_163_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
2758,FOODS_1_163_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
20002,FOODS_3_827_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
20003,FOODS_3_827_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
20004,FOODS_3_827_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
20005,FOODS_3_827_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [21]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [23]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [24]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 8.502024291497975 %
Percentage of unexpected values of WMAPE is: 8.502024291497975 %
Percentage of unexpected values of SMAPE is: 9.31174089068826 %
Percentage of unexpected values of MAPE is: 8.502024291497975 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [25]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [26]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE      WMAPE       SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                                
0.1   0.1   0.879108  76.825937   84.370440  74.304170  3.458881  4.486349
      0.2   0.872556  76.157898   84.828735  76.438480  3.466109  4.510896
      0.3   0.871834  76.085841   85.480559  78.171785  3.476305  4.538600
      0.4   0.873771  76.238622   86.316522  79.828024  3.493034  4.568002
      0.5   0.877766  76.587135   87.373262  81.602455  3.515109  4.600390
...              ...        ...         ...        ...       ...       ...
0.9   0.5   0.936647  81.089096   94.759643  88.176189  3.839691  5.191508
      0.6   0.942327  81.580084   95.825533  89.592123  3.867355  5.233500
      0.7   0.949720  82.240303   97.229238  91.240443  3.899689  5.280261
      0.8   0.959167  83.114008   99.162782  93.269739  3.937728  5.333280
      0.9   0.970562  84.183724  101.847990  95.716478  3.981439  5.394319

[81 rows x 6 columns]

In [27]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.2, 0.3)
The optimum Alpha and Beta based on Mean WMAPE is: (0.3, 0.3)
The optimum Alpha and Beta based on Mean SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAPE is: (0.2, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.2, 0.1)
The optimum Alpha and Beta based on Mean RMSE is: (0.2, 0.1)


### Get MEDIAN metrics value of each Alpha

In [28]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE    WMAPE    SMAPE     MAPE       MAE      RMSE
Alpha Beta                                                         
0.1   0.1   0.801057  68.0428  76.5185  67.5169  2.538023  3.189386
      0.2   0.807713  68.7120  76.2754  69.3673  2.539716  3.213315
      0.3   0.816356  69.0335  76.9504  69.9901  2.559804  3.216181
      0.4   0.822193  70.0070  78.2383  71.9625  2.568076  3.304086
      0.5   0.828819  70.8736  78.7228  72.5953  2.599223  3.358952
...              ...      ...      ...      ...       ...       ...
0.9   0.5   0.929238  79.7752  88.3708  80.5901  2.867149  3.679061
      0.6   0.933865  79.7744  90.1178  81.2092  2.897595  3.778457
      0.7   0.938470  80.0597  91.3907  82.2390  2.928711  3.844779
      0.8   0.949512  81.3439  92.9242  84.5286  2.975895  3.943671
      0.9   0.962108  82.7603  95.8325  86.9989  2.994496  4.004864

[81 rows x 6 columns]

In [29]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of WMAPE is: (0.2, 0.1)
The optimum Alpha and Beta based on Median of SMAPE is: (0.1, 0.2)
The optimum Alpha and Beta based on Median of MAPE is: (0.2, 0.1)
The optimum Alpha and Beta based on Median of MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of RMSE is: (0.1, 0.1)


# Run Croston model after decide best paramaters

In [30]:
best_alpha = 0.2
best_beta = 0.1

In [31]:
def CrostonTSB_test_data(best_alpha, best_beta, sales_pattern,
                         start_train_date, end_train_date, 
                         n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [32]:
start = timer()

pattern_df_result_best_params = CrostonTSB_test_data(best_alpha, best_beta, sales_erratic, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.05133679465000114 minutes


In [33]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [34]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_2_validation,1.011363,59.5408,66.5184,99.4842,3.189684,3.591609
1,FOODS_1_018_CA_2_validation,1.152603,81.1596,76.4981,129.0491,3.014501,3.486185
2,FOODS_1_018_CA_3_validation,0.789037,61.7987,64.4368,91.3220,2.427807,3.216758
3,FOODS_1_021_CA_1_validation,0.939043,80.6614,86.3003,151.1917,4.839684,5.835284
4,FOODS_1_021_CA_2_validation,0.893424,62.1388,64.4879,68.9061,2.130473,2.726424
...,...,...,...,...,...,...,...
242,FOODS_3_809_CA_4_validation,0.663627,107.2012,126.2847,38.8588,0.765723,0.928542
243,FOODS_3_811_CA_2_validation,1.014032,74.9755,93.7652,51.3936,3.588112,3.988278
244,FOODS_3_818_CA_2_validation,1.285912,109.8312,130.1196,68.6230,2.275076,2.783142
245,FOODS_3_822_CA_3_validation,0.655476,35.6914,39.8113,26.7657,2.268956,2.974001


In [35]:
df_result_final.to_csv('CrostonTSB_Erratic_Test_Data.csv')